In [1]:
using HorizonSideRobots

In [2]:
a = Robot("untitled.sit",animate=true);

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\Alex\.julia\packages\Makie\Qvk4f\src\scenes.jl:220


In [3]:
function inverse(side)
    if side == Ost
        return West
    end
    if side == West
        return Ost
    end
    if side == Sud
        return Nord
    end
    if side == Nord
        return Sud
    end
end

inverse (generic function with 1 method)

In [4]:
function along!(r, side)
    while !isborder(r, side)
        move!(r, side)
    end
end

along! (generic function with 1 method)

In [5]:
function try_move!(r, side)
    if !isborder(r, side)
        move!(r, side)
        return true
    end
    return false
end

try_move! (generic function with 1 method)

In [6]:
function HorizonSideRobots.move!(r, side, n_steps)
    steps_done = 0
    for _ in range(1, n_steps)
        if try_move!(r, side)
            steps_done += 1
        end
    end
    return steps_done
end

In [7]:
function mark_first_line!(r)
    putmarker!(r)
    steps_ost = 0
    while !isborder(r, Ost)
        steps_done = move!(r, Ost, 2)
        if steps_done == 2
            putmarker!(r)
        end
        steps_ost += steps_done
    end
    move!(r, West, steps_ost)
    steps_west = 0
    while !isborder(r, West)
        steps_done = move!(r, West, 2)
        if steps_done == 2
            putmarker!(r)
        end
        steps_west += steps_done
    end
    return steps_west
end

mark_first_line! (generic function with 1 method)

In [8]:
function to_start!(r)
    marked = ismarker(r)
    arr = []
    while !(isborder(r, Sud) && isborder(r, West))
        steps_sud = 0
        steps_west = 0
        if try_move!(r, Sud)
            steps_sud += 1
            marked = !marked
        end
        if try_move!(r, West)
            steps_west += 1
            marked = !marked
        end
        append!(arr, steps_sud)
        append!(arr, steps_west)
    end
    return arr, marked
end

to_start! (generic function with 1 method)

In [9]:
function mark_any_line!(r, side, first_marked)
    if first_marked
        putmarker!(r)
    else
        if try_move!(r, side)
            putmarker!(r)
        end
    end
    while !isborder(r, side)
        moved = move!(r, side, 2)
        if moved == 2
            putmarker!(r)
        end
    end
    along!(r, inverse(side))
end

mark_any_line! (generic function with 1 method)

In [10]:
function mark_all!(r, marked, side, ort_side)
    mark_any_line!(r, side, marked)
    while !isborder(r, ort_side)
        marked_2 = !ismarker(r)
        move!(r, ort_side)
        mark_any_line!(r, side, marked_2)
    end
end

mark_all! (generic function with 1 method)

In [11]:
function main!(r)
    steps_west_fl = mark_first_line!(r)
    arr, marked = to_start!(r)
    mark_all!(r, marked, Nord, Ost)
    along!(r, Nord)
    marked = ismarker(r)
    mark_all!(r, marked, Sud, West)
    along!(r, Sud)
    marked = ismarker(r)
    mark_all!(r, marked, Ost, Nord)
    along!(r, Sud)
    along!(r, Ost)
    marked = ismarker(r)
    mark_all!(r, marked, West, Nord)
    along!(r, Sud)
    along!(r, West)
    side = Ost
    for i in range(-length(arr), -1)
        move!(r, side, arr[abs(i)])
        if side == Ost
            side = Nord
        else
            side = Ost
        end
    end
    move!(r, Ost, steps_west_fl)
end

main! (generic function with 1 method)

In [12]:
main!(a)